# Imports

In [ ]:
# system and data handling packages
import os
import sys
import json
import csv
import collections as col

# matplotlib packages
%matplotlib inline
import matplotlib as mpl
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.pyplot as plt
# change figure fonts
from matplotlib import rcParams

# import jsci, CT's enhanced json stream write package
import jsci
from jsci import WriteStream as jsciwrite
from jsci import Coding as jscicoding

# numpy
import numpy as np
from scipy.interpolate import interp1d
import scipy as sp
from scipy import stats
from scipy.optimize import curve_fit
import scipy.integrate as spint
from numpy import fft as npft

# import Kitaev Honeycomb package
import kithcmb
from kithcmb import ThermalGradient as vs

In [ ]:
plt.style.use('prettyfigs')

In [ ]:
print(mpl.get_configdir())

In [ ]:
# These are the "Tableau 20" colors as RGB.       
tableau20 = [(31, 119, 180),(44, 160, 44),(255, 127, 14), 
             (214, 39, 40),  
             (148, 103, 189),(140, 86, 75),
             (227, 119, 194),(127, 127, 127),
             (188, 189, 34),(23, 190, 207),]   
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)  

# Read data

In [ ]:
data_paths = []
data_paths.append( os.pardir + '/data/2016-11-10_2111_IG' )
data_paths.append( os.pardir + '/data/2016-11-11_1356_IG' )
data_paths.append( os.pardir + '/data/2016-11-11_2011_IG' )
data_paths.append( os.pardir + '/data/2016-11-11_2015_IG' )

In [ ]:
L_set = [12,14,16,18,20]
max_t = '200.0'
dt = '0.1'
T_set = np.arange(1,20)
J = '1.0'
K = '0.1'
#
temps = np.array([ 10**( -2 + 3./20 * T ) for T in T_set ])
times = np.arange( -float(max_t),float(max_t),float(dt) )
freqs = np.fft.fftfreq(len(times),float(dt))

In [ ]:
def read_data( L_,T_ ):
    for dpath in data_paths:
        try:
            filename = 'response-current_NV_L'+str(L_)+'_T'+str(T_)+'_tmax'+max_t+'_dt'+dt+'_J'+J+'_K'+K+'.json'
            with open( dpath+'/'+filename, 'r') as file:
                data = json.load( file, cls=jscicoding.NumericDecoder )
            return data
        except IOError:
            pass
        except ValueError:
            # print 'corrupt data file ',dpath+'/'+filename
            pass
    raise IOError

In [ ]:
def load_data( L_,T_ ):
    # read the data file
    try:
        data = read_data(L_,T_)
    except IOError:
        print 'could not find file for ',(L,T)
        raise IOError        
    
    # rearrange the data to have the time series in an easier format
    data['xcurr'] = []
    data['zcurr'] = []
    for t_series in data['time_series']:
        data['xcurr'].append(t_series['x']/L_**2)
        data['zcurr'].append(t_series['z']/L_**2)

    data['xcurr'] = np.lib.pad( data['xcurr'], (len(times)-len(data['xcurr']),0), 'constant' )
    data['zcurr'] = np.lib.pad( data['zcurr'], (len(times)-len(data['zcurr']),0), 'constant' )
    
    # compute the Fourier transforms
    data['xspectrum'] = -1j*(2*np.pi) * np.fft.fft(data['xcurr'])
    data['zspectrum'] = -1j*(2*np.pi) * np.fft.fft(data['zcurr'])
        
    return data

In [ ]:
data = {}
for L in L_set:
    for T in T_set:
        data[L,T] = load_data(L,T)

# Analyse

In [ ]:
for L in L_set:
    print 'L : ',L
    for T in T_set:
        print 'T : ',T
        fig,ax = plt.subplots()
        plt.plot( times,data[L,T]['xcurr'] )
        plt.xlabel(r'$t$')
        plt.ylabel(r'$I_x$')

        #plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
        plt.show()

In [ ]:
for L in L_set:
    print 'L : ',L
    for T in T_set:
        print 'T : ',T
        fig,ax = plt.subplots()
        plt.plot([],[])
        plt.plot( times,data[L,T]['zcurr'] )
        plt.xlabel(r'$t$')
        plt.ylabel(r'$I_z$')

        #plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
        plt.show()

In [ ]:
display = 'xspectrum'

In [ ]:
for L in [18]:
    print 'L : ',L
    for T in T_set:
        print 'T : ',T
        fig,ax = plt.subplots()
        plt.plot( freqs,data[L,T][display].real,',-',freqs,data[L,T][display].imag,',-' )
        plt.xlabel(r'$\omega$')
        plt.ylabel(r'$\mathcal{F} \, [I_x]$')

        #plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
        plt.show()

In [ ]:
for L in [18]:
    print 'L : ',L
    for T in T_set:
        print 'T : ',T
        fig,ax = plt.subplots()
        plt.plot( freqs,freqs*data[L,T][display].real,',-',freqs,freqs*data[L,T][display].imag,',-', linewidth=0.5, markersize=2.5 )
        plt.xlabel(r'$\omega$')
        plt.ylabel(r'$\omega \, \mathcal{F} \, [I_x]$')

        #plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
        plt.show()

In [ ]:
fig,ax = plt.subplots()
for L in L_set:
    L_series = []
    for T in T_set:
        L_series.append( (data[L,T]['xspectrum'].imag)[0]/np.pi )
    plt.plot( temps,L_series,'o-',label=r'$L=$'+str(L) )

plt.plot( temps,[0. for x in temps], 'k--' )
plt.xlabel(r'$T$')
plt.ylabel(r'$\mathcal{F} \, [I_x]( \omega = 0 )$')
plt.legend(loc=4,fontsize=16)
#plt.xlim([0,2])
#plt.ylim([0.3,2.5])

#plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
plt.show()

In [ ]:
fig,ax = plt.subplots()
for L in L_set:
    L_series = []
    for T in T_set:
        L_series.append( -1*(data[L,T]['zspectrum'].imag)[0]/np.pi )
    plt.plot( temps,L_series,'o-',label=r'$L=$'+str(L) )

plt.xlabel(r'$T$')
plt.ylabel(r'$\mathcal{F} \, [I_z](\omega = 0 )$')
plt.legend(loc=4,fontsize=16)
#plt.xlim([0,1])
#plt.ylim([0,3.])

#plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
plt.show()

In [ ]:
fig,ax = plt.subplots()
runtimes = []
for L in L_set:
    max_runtime = 0.
    for T in T_set:
        max_runtime = max([ data[L,T]['run_time'], max_runtime ])
    runtimes.append(max_runtime / 3600.)
plt.plot( L_set,runtimes,'o-')

plt.xlabel(r'$L$')
plt.ylabel('run time / hours')
plt.legend(loc=4,fontsize=16)

#plt.savefig( 'long_curr-curr-'+correl+'.pdf',  bbox_extra_artist=(leg,), bbox_inches='tight', transparency=True )
plt.show()